<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#组合训练数据" data-toc-modified-id="组合训练数据-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>组合训练数据</a></span></li></ul></div>

# 组合训练数据

得到单个样例的处理结果之后, 还需要将他们组织成batch, 然后再提供给神经网络的输入层。

TensorFlow 提供了以下四个函数来讲单个的样例组织成一个batch形式输出。

1. `tf.train.batch()`
2. `tf.train.shuffle_batch()`
3. `tf.train.batch_join()`
4. `tf.train.shuffle_batch_join()`

这个四个函数都会生成一个队列, 队列的入队操作是生成单个样例的方法, 而每次出队操作得到是一个batch的样例;其中, 带有`shuffle`的表示要把数据打乱后输出;

上述函数除了可以将单个数据整理成输入batch, 也提供了并行化处理输入数据的方法(可以指定多个线程同时进行入队操作). `tf.train.batch` 和 `tf.train.shuffle_bathc`是一致的, 以`tf.train.shuffle`为例, 当指定`num_threads`参数大于1时, 多个线程会同时读取文件中的不同样例进行并行化预处理。

如果需要多个线程处理不同文件中的样例时, 可以使用`tf.train.shuffle_batch_join`, 此函数会从文件队列中获取不同的文件分配给不同的线程。

一般来说, 输入文件队列是通过`tf.strain.string_input_producer`函数生成, 这个函数会平均分配文件以保证不同文件中数据会被尽量平均使用.

**join与非jion函数的区别**

`tf.train.shuffle_batch` 和 `tf.train.shuffle_batch_join`函数可以完成多线程并行的方式来进行数据预处理，但是他们各有优劣. 对于`tf.train.shuffle_batch`函数, 不同线程会读取同一个文件, 如果一个文件中的样例比较相似(比如都属于同一个类别), 那么神经网络的训练效果有可能会受到影响. 所以在使用`tf.train.shuffle_batch`函数时, 需要尽量将同一个`TFRecords`文件中的样例随机打乱. 而使用`tf.train.shuffle_batch_join`函数时, 不同的线程会读取不同文件. 如果读取数据的线程总数比总文件数还大, 那么多个线程就会读取同一个文件中相似部分的数据. 而且, 多个线程读取多个文件可能导致过多的硬盘寻址, 从而使得读取速率降低.

In [4]:
# 加载库文件
import tensorflow as tf

In [5]:
tf.reset_default_graph()

# 获取文件列表
files = tf.train.match_filenames_once("../../../other/test/data.tfrecords-*")

# 创建文件输入队列
filename_queue = tf.train.string_input_producer(files, shuffle=False)

# 申请一个reader
reader = tf.TFRecordReader()

# 读取文件中的样例
_, serialized_example = reader.read(filename_queue)

# 把样例解析为特征

feature_i = tf.FixedLenFeature([], tf.int64)
feature_j = tf.FixedLenFeature([], tf.int64)
feature_map = {'i': feature_i, 'j': feature_j}
featues = tf.parse_single_example(serialized_example, features=feature_map)

# 提取数据
example, label = featues['i'], featues['j']

# 设置参数(batch和容量)
batch_size = 3
capacity = 1000 + 3 * batch_size

以下代码展示了 `tf.train.batch()` 的用法.

In [6]:
# 使用tf.train.batch函数来组合样例
# [example, label]参数给出需要组合的元素
# 一般example和label给出了分别代表训练样本和标签
# batch_size给出了batch中样例的大小
# capacity给出了队列中的最大容量, 当队列长度等于容量时, TensorFlow将暂停入队操作, 
# 只等待元素出队, 当元素个数小于容量时, TensorFlow将自动重新启动入队操作
example_batch, label_batch = tf.train.batch([example, label], batch_size=batch_size, capacity=capacity)

with tf.Session() as sess:
    sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
    
    # 打印文件列表
    print("files: ", files)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    # 获取打印之后的样例;在真实问题中, 这个输出一般会作为神经网路的输入
    for i in range(2):
        cur_example_batch, cur_label_batch = sess.run([example_batch, label_batch])
        print(cur_example_batch, cur_label_batch)
        
    coord.request_stop()
    coord.join(threads)

files:  <tf.Variable 'matching_filenames:0' shape=<unknown> dtype=string_ref>
[0 0 0] [0 1 2]
[0 0 1] [3 4 0]


以下代码展示了 `tf.train.shuffle_batch()` 的用法.

`tf.train.shuffle_batch` 的用法大部分与 `tf.train.batch` 的用法一致, 但是`min_after_dequeue` 参数是`tf.train.shuffle_batch` 函数特有的。`min_after_dequeu`e参数限制了出队时队列中元素的最少个数。当队列中元素太少时, 随机打乱样例顺序的作用就不大。所以 `tf.train.shuffle_batch`函数提供了限制出队时最少元素个数来保证随机打乱顺序的作用。当出队函数被调用但是队列中元素不够时, 出队操作将等待更多元素入队才会完成。如果 `min_after_dequeue` 参数被设定, `capacity`也应相应调整来满足性能的需求.

In [7]:
example_batch, label_batch = tf.train.shuffle_batch(
    [example, label], 
    batch_size=batch_size, 
    capacity=capacity, 
    min_after_dequeue=30)

with tf.Session() as sess:
    sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
    
    # 打印文件列表
    print("files: ", files)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    # 获取打印之后的样例;在真实问题中, 这个输出一般会作为神经网路的输入
    for i in range(2):
        cur_example_batch, cur_label_batch = sess.run([example_batch, label_batch])
        print(cur_example_batch, cur_label_batch)
        
    coord.request_stop()
    coord.join(threads)


files:  <tf.Variable 'matching_filenames:0' shape=<unknown> dtype=string_ref>
[6 0 3] [0 4 4]
[3 3 1] [4 1 4]
